## Setting Up the Dataset

In [1]:
import dataset_generator
import pandas as pd
import numpy as np

In [5]:
dataframe = pd.read_csv('data_train.csv')

In [14]:
upper = dataframe[dataframe['A'] >= 0.5].min()['A']
lower = dataframe[dataframe['A'] <  0.5].max()['A']

print(f'{lower} <= boundary <= {upper}')

0.4830102287575435 <= boundary <= 0.5101910542728111


The actual rule is: $A \ge 0.5 \implies target = 1$

The model should predict the boundary as a value between `lower` and `upper`, inclusive.

## Using an `anchor` Explainer

In [1]:
from anchor import utils
import tensorflow as tf

ImportError: cannot import name 'utils' from 'anchor' (/home/tiagovargas/.local/lib/python3.10/site-packages/anchor/__init__.py)

In [16]:
anchor_dataset = utils.load_csv_dataset('data_train.csv', target_idx=-1)

Training the model

In [22]:
data_train = pd.read_csv('test_data.csv').to_numpy()
x_train = data_train[:, :-1]
y_train = data_train[:, -1]

In [67]:
keras_model = tf.keras.Sequential(
	[tf.keras.layers.Input(shape=x_train.shape[1])]
)

In [68]:
number_of_classes = 2
number_of_neurons = 20
number_of_hidden_layers = 4

In [70]:
for _ in range(number_of_hidden_layers):
	keras_model.add(
		tf.keras.layers.Dense(units=number_of_neurons, activation='relu')
	)

In [71]:
keras_model.add(
	tf.keras.layers.Dense(units=number_of_classes, activation='softmax')
)

In [72]:
keras_model.compile(
	optimizer=tf.keras.optimizers.Adam(),
	loss='categorical_crossentropy',
	metrics=['accuracy']
)

In [76]:
es = tf.keras.callbacks.EarlyStopping(patience=10)
ck = tf.keras.callbacks.ModelCheckpoint(
	f'model_{number_of_hidden_layers}layers.h5',
	save_best_only=True,
	monitor='val_accuracy'
)

In [77]:
y_train_ohe = tf.keras.utils.to_categorical(y_train, number_of_classes)

In [78]:
keras_model.fit(
	x_train,
	y_train_ohe,
	batch_size=4,
	epochs=100,
	callbacks=[ck, es]
)

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 1.0000
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 11/